In [31]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from unidecode import unidecode
from tqdm.contrib.concurrent import thread_map
from functools import partial
import itertools
from concurrent.futures import ThreadPoolExecutor
from dateutil import parser
import os

In [2]:
feeds_url = 'https://www.demorgen.be/sitemaps/news.xml'
page_divs = requests.get(feeds_url).content
soup = BeautifulSoup(page_divs, "html.parser")
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:news="http://www.google.com/schemas/sitemap-news/0.9">
 <url>
  <loc>
   https://www.demorgen.be/tv-cultuur/wim-lybaert-over-zijn-jonge-jaren-mijn-omgeving-gaf-me-alles-wat-ik-verlangde-ik-hoefde-niet-te-dromen~b85ff213/
  </loc>
  <lastmod>
   2023-08-08T08:00:07.009Z
  </lastmod>
  <news:news>
   <news:publication>
    <news:name>
     De Morgen
    </news:name>
    <news:language>
     nl
    </news:language>
   </news:publication>
   <news:publication_date>
    2023-08-08T08:00:07.009Z
   </news:publication_date>
   <news:title>
    Wim Lybaert over zijn jonge jaren: ‘Mijn omgeving gaf me alles wat ik verlangde, ik hoefde niet te dromen’
   </news:title>
  </news:news>
 </url>
 <url>
  <loc>
   https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-zelensky-tegenoffensief-vordert-langzamer-dan-verwacht~b38bed0a/
  </loc>
  <lastmod>
   2023-08-08T07:49:40.375Z
  </lastmod>
  <

/home/mahmoud/Desktop/Training/Data-Tank-project/env/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [3]:
links = []
all_links = soup.find_all("loc")
for link in all_links:
    links.append(link.text)
links

['https://www.demorgen.be/tv-cultuur/wim-lybaert-over-zijn-jonge-jaren-mijn-omgeving-gaf-me-alles-wat-ik-verlangde-ik-hoefde-niet-te-dromen~b85ff213/',
 'https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-zelensky-tegenoffensief-vordert-langzamer-dan-verwacht~b38bed0a/',
 'https://www.demorgen.be/snelnieuws/onderzoek-naar-ontploffing-molotovcocktail-aan-voordeur-woning-in-aalst~be81c981/',
 'https://www.demorgen.be/snelnieuws/amerikaanse-diplomaat-op-bezoek-bij-coupplegers-niger-die-nieuwe-premier-benoemen~b3332932/',
 'https://www.demorgen.be/nieuws/trump-en-achterban-openen-jacht-op-de-biden-crime-family-met-zoon-hunter-als-belangrijkste-doelwit~bdaaa836/',
 'https://www.demorgen.be/snelnieuws/ontbossing-in-braziliaans-amazonewoud-vertraagt-klimaatexperts-kijken-hoopvol-uit-naar-bijeenkomst-met-zuid-amerikaanse-leiders~be3c0dda/',
 'https://www.demorgen.be/snelnieuws/eerste-spaanse-tourwinnaar-federico-bahamontes-op-95-jarige-leeftijd-overleden~bee780a3/',
 'https://www.demorge

In [4]:
print(links)


['https://www.demorgen.be/tv-cultuur/wim-lybaert-over-zijn-jonge-jaren-mijn-omgeving-gaf-me-alles-wat-ik-verlangde-ik-hoefde-niet-te-dromen~b85ff213/', 'https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-zelensky-tegenoffensief-vordert-langzamer-dan-verwacht~b38bed0a/', 'https://www.demorgen.be/snelnieuws/onderzoek-naar-ontploffing-molotovcocktail-aan-voordeur-woning-in-aalst~be81c981/', 'https://www.demorgen.be/snelnieuws/amerikaanse-diplomaat-op-bezoek-bij-coupplegers-niger-die-nieuwe-premier-benoemen~b3332932/', 'https://www.demorgen.be/nieuws/trump-en-achterban-openen-jacht-op-de-biden-crime-family-met-zoon-hunter-als-belangrijkste-doelwit~bdaaa836/', 'https://www.demorgen.be/snelnieuws/ontbossing-in-braziliaans-amazonewoud-vertraagt-klimaatexperts-kijken-hoopvol-uit-naar-bijeenkomst-met-zuid-amerikaanse-leiders~be3c0dda/', 'https://www.demorgen.be/snelnieuws/eerste-spaanse-tourwinnaar-federico-bahamontes-op-95-jarige-leeftijd-overleden~bee780a3/', 'https://www.demorgen.be/sn

In [5]:
for link in links:
    link_divs = requests.get(link).content
    soup_link = BeautifulSoup(link_divs, "html.parser")
    link_img = soup_link.find_all('img')
    print(len(link_img))
    print(link)

26
https://www.demorgen.be/tv-cultuur/wim-lybaert-over-zijn-jonge-jaren-mijn-omgeving-gaf-me-alles-wat-ik-verlangde-ik-hoefde-niet-te-dromen~b85ff213/
34
https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-zelensky-tegenoffensief-vordert-langzamer-dan-verwacht~b38bed0a/
22
https://www.demorgen.be/snelnieuws/onderzoek-naar-ontploffing-molotovcocktail-aan-voordeur-woning-in-aalst~be81c981/
24
https://www.demorgen.be/snelnieuws/amerikaanse-diplomaat-op-bezoek-bij-coupplegers-niger-die-nieuwe-premier-benoemen~b3332932/
22
https://www.demorgen.be/nieuws/trump-en-achterban-openen-jacht-op-de-biden-crime-family-met-zoon-hunter-als-belangrijkste-doelwit~bdaaa836/
23
https://www.demorgen.be/snelnieuws/ontbossing-in-braziliaans-amazonewoud-vertraagt-klimaatexperts-kijken-hoopvol-uit-naar-bijeenkomst-met-zuid-amerikaanse-leiders~be3c0dda/
24
https://www.demorgen.be/snelnieuws/eerste-spaanse-tourwinnaar-federico-bahamontes-op-95-jarige-leeftijd-overleden~bee780a3/
112
https://www.demorgen.be/

In [6]:
special_link = 'https://www.demorgen.be/meningen/van-9-tot-5~bbd332cb/'

link_divs = requests.get(special_link).content
soup_link = BeautifulSoup(link_divs, "html.parser")
link_img = soup_link.find_all('span', {"class": "artstyle__labels__label"})
# for span in link_img:
#     print(span.text)
#print(soup_link.prettify())
print(link_img[0].text)

Cartoon


In [7]:

script = soup_link.find('script', {"type": "application/ld+json"})
data = json.loads(script.text, strict=False)
#print(data['@graph'])
# for serial in data['@graph']['author']:
#     print (serial)
print(data['@graph'][1]['datePublished'])
time = data['@graph'][1]['datePublished']
print(time)

2023-08-07T12:51:00+0200
2023-08-07T12:51:00+0200


In [8]:
link_divs = requests.get(link).content
soup_link = BeautifulSoup(link_divs, "html.parser")
# print(soup_link.prettify())
print(link)

https://www.demorgen.be/nieuws/leeftijd-grootste-bron-van-discriminatie-op-arbeidsmarkt-110-sollicitaties-stuurde-ik-uiteindelijk-nam-ik-werk-aan-op-140-km-van-huis~b0bfbacb5/


In [9]:
p_tags = soup_link.find_all('p')
p_tags

[<p class="share-modal__copy"> <input class="share-modal__copy-input fjs-share-modal-copy-input" type="hidden" value="https://www.demorgen.be/cs-b0bfbacb5"/> <a class="share-modal__copy-link fjs-share-modal-copy-link" data-gtm="share / top / copylink" href="https://www.demorgen.be/cs-b0bfbacb5"> <svg class="share-modal__copy-icon" height="10" width="20"> <path d="M1.9 5c0-1.7 1.4-3.1 3.1-3.1h4V0H5a5 5 0 0 0 0 10h4V8.1H5A3.1 3.1 0 0 1 1.9 5zM6 6h8V4H6v2zm9-6h-4v1.9h4a3.1 3.1 0 0 1 0 6.2h-4V10h4a5 5 0 0 0 0-10z" fill="#000" fill-rule="nonzero"></path> </svg> <span class="share-modal-copy-link-text fjs-share-modal-copy-link-text" data-click-value="Link gekopieerd"> Kopieer de link van het artikel </span> </a> </p>,
 <p class="artstyle__intro artstyle__intro--type-luxe fu--type-luxe" data-element-id="33a3-1bb4-908f-aa79-0d32-087e-0e9d-ea93">Nu uit onderzoek nog maar eens blijkt dat leeftijd de belangrijkste factor is voor discriminatie op de arbeidsmarkt, vertelt Luc Corremans (57) hoe moe

In [10]:
p_title = soup_link.find_all('title')
Title = unidecode(p_title[0].text.split('|')[0].strip())

Title

"Leeftijd grootste bron van discriminatie op arbeidsmarkt: '110 sollicitaties stuurde ik. Uiteindelijk nam ik werk aan op 140 km van huis'"

In [11]:
paragraphs = ''
p_intro = soup_link.find_all('p', {"class": "artstyle__intro"})
print(unidecode(p_intro[0].text))
paragraphs  += unidecode(p_intro[0].text.strip())

Nu uit onderzoek nog maar eens blijkt dat leeftijd de belangrijkste factor is voor discriminatie op de arbeidsmarkt, vertelt Luc Corremans (57) hoe moeilijk hij aan nog een nieuwe job raakte. Magda Duerinckx van Acerta geeft tips voor oudere sollicitanten. 'Probeer niet te concurreren met de jeugd op het vlak van snelheid.'
  


In [12]:
p_tags = soup_link.find_all('p', {"class": "artstyle__paragraph"})
p_tags

[<p class="artstyle__paragraph artstyle__paragraph--drop-cap fu--drop-cap artstyle__paragraph--drop-cap-l fu--drop-cap-l" data-element-id="a82e-aba1-8878-9ec1-39e7-f2d2-4801-468a">
       Luc was de 55 al gepasseerd toen zijn job als verkoop- en marketingmanager na een overname werd geschrapt. “Ik begreep dat het niet makkelijk zou zijn om nog iets anders te vinden. Je wordt er niet jonger op. Om geen kansen te laten passeren, ben ik direct beginnen te zoeken, in plaats van het eerst drie à vier maanden rustig aan te doen.”
     </p>,
 <p class="artstyle__paragraph" data-element-id="2222-a741-47a5-dd39-087c-3aa6-db2c-29c8">
       Vacatures scannen, uitzoeken over welk bedrijf het gaat en of de job wel op zijn lijf was geschreven, en dan motivatiebrieven uitwerken: Luc zegt dat hij daar zeker vier uur per dag mee bezig was. “In mijn sollicitaties probeerde ik vooroordelen over mijn leeftijd, toen 56 jaar, te counteren. Veel bedrijven vrezen dat je niet mee kan met de digitalisering. Ik

In [13]:
for parag in p_tags:
    paragraphs += unidecode(parag.text.strip())
print(paragraphs)

Nu uit onderzoek nog maar eens blijkt dat leeftijd de belangrijkste factor is voor discriminatie op de arbeidsmarkt, vertelt Luc Corremans (57) hoe moeilijk hij aan nog een nieuwe job raakte. Magda Duerinckx van Acerta geeft tips voor oudere sollicitanten. 'Probeer niet te concurreren met de jeugd op het vlak van snelheid.'Luc was de 55 al gepasseerd toen zijn job als verkoop- en marketingmanager na een overname werd geschrapt. "Ik begreep dat het niet makkelijk zou zijn om nog iets anders te vinden. Je wordt er niet jonger op. Om geen kansen te laten passeren, ben ik direct beginnen te zoeken, in plaats van het eerst drie a vier maanden rustig aan te doen."Vacatures scannen, uitzoeken over welk bedrijf het gaat en of de job wel op zijn lijf was geschreven, en dan motivatiebrieven uitwerken: Luc zegt dat hij daar zeker vier uur per dag mee bezig was. "In mijn sollicitaties probeerde ik vooroordelen over mijn leeftijd, toen 56 jaar, te counteren. Veel bedrijven vrezen dat je niet mee ka

In [14]:
from unidecode import unidecode
x = unidecode(x)
x

NameError: name 'x' is not defined

In [ ]:
data['url-parent'][2]

'https://www.demorgen.be/snelnieuws/zweden-scherpt-grenscontroles-aan-na-koranverbrandingen~bd4dfe35/'

In [ ]:
def get_links(feeds_url):
    page_divs = requests.get(feeds_url).content
    soup = BeautifulSoup(page_divs, "html.parser")
    links = []
    all_links = soup.find_all("loc")
    for link in all_links:
        links.append(link.text)
    return links
    
feeds_url = 'https://www.demorgen.be/sitemaps/news.xml'
links  = get_links(feeds_url)

/home/mahmoud/Desktop/Training/Data-Tank-project/env/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [17]:
def get_title(soup): # Extract title 
    return soup.find_all('title')[0].text.split('|')[0].strip()

def get_text(soup): # Extract the article's paragraphs
    paragraphs = ''
    p_intro = soup.find_all('p', {"class": "artstyle__intro"}) # article intro
    if p_intro:
        paragraphs  += p_intro[0].text.strip()
    p_tags = soup.find_all('p', {"class": "artstyle__paragraph"})
    for parag in p_tags:
        paragraphs += parag.text.strip() # adding article's paragraphs
    return paragraphs

def get_date(soup): # Extract the published date
    script = soup.find('script', {"type": "application/ld+json"})
    data = json.loads(script.text, strict=False)
    return data['@graph'][1]['datePublished']

def get_links(feeds_url): # Extract links from news feed
    page_divs = requests.get(feeds_url).content
    soup = BeautifulSoup(page_divs, "html.parser")
    links = []
    all_links = soup.find_all("loc")
    for link in all_links:
        links.append(link.text)
    return links
feeds_url = 'https://www.demorgen.be/sitemaps/news.xml'

# links  = get_links(feeds_url)
source_url = 'https://www.demorgen.be/snelnieuws/junta-sluit-het-luchtruim-boven-niger-vanwege-dreigende-interventie-west-afrikaanse-landen~b8e6f72d/'
session = requests.Session()
page_html = session.get(source_url).content
soup = BeautifulSoup(page_html, "html.parser")
page_type = soup.find_all('span', {"class": "artstyle__labels__label"})
if page_type:
    if (page_type[0].text == 'Cartoon'):
                print('Cartoon page ! ')
    else:
        article_title = get_title(soup)
        article_text = get_text(soup)
        published_date = get_date(soup)
        row = [[source_url,article_title,article_text,published_date,'NL']]
        df = pd.DataFrame(row,columns = ["Source_url", "article_title", "article_text","published_date","language"])

        #dnew_row = {"Source_url":source_url, "article_title":article_title, "article_text":article_text,"published_date":published_date,"language":'NL'}
        #df.loc[len(df)] = new_row
else:
     article_title = get_title(soup)
     article_text = get_text(soup)
     published_date = get_date(soup)
     row = [[source_url,article_title,article_text,published_date,'NL']]
     df = pd.DataFrame(row,columns = ["Source_url", "article_title", "article_text","published_date","language"])

In [18]:
df

,Source_url,article_title,article_text,published_date,language
0,https://www.demorgen.be/snelnieuws/junta-sluit...,Junta sluit het luchtruim boven Niger vanwege ...,De militaire junta in Niger heeft het luchtrui...,2023-08-07T01:02:00+0200,NL


In [42]:
def get_title(soup): # Extract title 
    return soup.find_all('title')[0].text.split('|')[0].strip()

def get_text(soup): # Extract the article's paragraphs
    paragraphs = ''
    p_intro = soup.find_all('p', {"class": "artstyle__intro"}) # article intro
    if p_intro:
        paragraphs  += p_intro[0].text
    p_tags = soup.find_all('p', {"class": "artstyle__paragraph"})
    for parag in p_tags:
        paragraphs += parag.text # adding article's paragraphs
    return paragraphs

def get_date(soup): # Extract the published date
    script = soup.find('script', {"type": "application/ld+json"})
    data = json.loads(script.text, strict=False)
    return data['@graph'][1]['datePublished']


def get_links(feeds_url): # Extract links from news feed
    page_divs = requests.get(feeds_url).content
    soup = BeautifulSoup(page_divs, "html.parser")
    links = []
    all_links = soup.find_all("loc")
    for link in all_links:
        links.append(link.text)
    return links

def get_article(url, session):

    try:
        page_html = session.get(url).content
        soup = BeautifulSoup(page_html, "html.parser")
        page_type = soup.find_all('span', {"class": "artstyle__labels__label"})
        page_blog = soup.find_all('span', {"class": "artstyle__labels__live"})
        if page_type:
            if (page_type[0].text == 'Cartoon'):
                return 
            else:
                article_title = get_title(soup)
                article_text = get_text(soup)
                published_date = parser.parse(get_date(soup))
                row = [[url,article_text,published_date,article_title,'NL']]
                df = pd.DataFrame(row,columns = ["url","text","date","title","language"])
        else:
            if page_blog:
                return
            else:
                article_title = get_title(soup)
                article_text = get_text(soup)
                published_date = parser.parse(get_date(soup))
                row = [[url,article_text,published_date,article_title,'NL']]
                df = pd.DataFrame(row,columns = ["url", "text","date","title", "language"])    
        return df
    
    except Exception as e:
        print(type(e))
        print(e)
        return e

if __name__ == "__main__":

    feeds_url = 'https://www.demorgen.be/sitemaps/news.xml'
    links  = get_links(feeds_url)

    with requests.Session() as session:
        news_feed = pd.concat(df for df in thread_map(partial(get_article, session=session), links) 
                               if isinstance(df, pd.DataFrame))
    
news_feed_dict= news_feed.to_dict('records')
  
    
#    news_feed.to_csv("Demorgen.csv")






/home/mahmoud/Desktop/Training/Data-Tank-project/env/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
100%|██████████| 140/140 [00:18<00:00,  7.65it/s]

{'url': 'https://www.demorgen.be/snelnieuws/thuiswerkbedrijf-zoom-laat-werknemers-niet-langer-onbeperkt-op-afstand-werken~b5d63ce7/', 'text': 'Het Amerikaanse bedrijf Zoom, bekend van de software voor online vergaderen, herziet zijn aanpak. Personeelsleden die in de buurt wonen, worden verplicht om minstens twee dagen per week aanwezig te zijn op kantoor.\n  \n      Tijdens de coronapandemie raakte de term “zoomen” beroemd dankzij de vele online bijeenkomsten die met de software werden georganiseerd. Het Amerikaanse Zoom omarmt nu zelf een hybride werkmodel waarbij medewerkers balans vinden tussen thuiswerken en kantoorwerk.\n    \n      De nieuwe maatregel geldt voor alle werknemers die binnen 80 kilometer van kantoor wonen. Het bedrijf gelooft dat werken op kantoor efficiëntie bevordert, waardoor innovaties makkelijker kunnen worden doorgevoerd en gebruikers beter worden ondersteund.\n    \n      Ook andere bedrijven, zoals Google, Amazon en Disney, kiezen al een tijdje voor een hybr

In [43]:
news_feed

,url,text,date,title,language
0,https://www.demorgen.be/snelnieuws/thuiswerkbe...,"Het Amerikaanse bedrijf Zoom, bekend van de so...",2023-08-08 14:40:41+02:00,Thuiswerkbedrijf Zoom laat werknemers niet lan...,NL
0,https://www.demorgen.be/snelnieuws/lezersoproe...,In tien jaar tijd is het aantal spoedopnames d...,2023-08-08 14:40:00+02:00,Lezersoproep. Op welke manier hebben allergieë...,NL
0,https://www.demorgen.be/nieuws/op-de-amsterdam...,"We zijn in Amsterdam, het is vrijdagavond tien...",2023-08-08 14:00:25+02:00,Op de Amsterdamse Wallen: ‘Het zijn altijd vro...,NL
0,https://www.demorgen.be/beter-leven/van-koffie...,"Bent u verslaafd? ‘Absoluut niet!’, denkt u nu...",2023-08-08 13:46:00+02:00,"Koffie, suiker, alcohol: verslavingen loeren o...",NL
0,https://www.demorgen.be/snelnieuws/afscheid-va...,"Donderdag 10 augustus nemen familie, vrienden ...",2023-08-08 13:31:20+02:00,Afscheid van Tom Pintens donderdag in OLT Rivi...,NL
...,...,...,...,...,...
0,https://www.demorgen.be/tv-cultuur/heropende-k...,De zeventiende editie van de Museumnacht in An...,2023-08-06 16:16:11+02:00,Heropende KMSKA publiekstrekker tijdens Museum...,NL
0,https://www.demorgen.be/sport/alsof-ze-nooit-i...,Simone Biles (26) kan het nog. Zoveel toonde d...,2023-08-06 15:47:06+02:00,Alsof ze nooit is weggeweest: Simone Biles win...,NL
0,https://www.demorgen.be/snelnieuws/attenzione-...,De stem van de 57-jarige Monica Poli is wereld...,2023-08-06 14:40:31+02:00,‘Attenzione pickpocket!’ Italiaanse TikTok-ste...,NL
0,https://www.demorgen.be/snelnieuws/titelverded...,Voor de voetbalsters van de Verenigde Staten i...,2023-08-06 14:34:05+02:00,Titelverdediger Verenigde Staten na strafschop...,NL


In [44]:
for x in links:
    print(x)

https://www.demorgen.be/snelnieuws/thuiswerkbedrijf-zoom-laat-werknemers-niet-langer-onbeperkt-op-afstand-werken~b5d63ce7/
https://www.demorgen.be/snelnieuws/lezersoproep-op-welke-manier-hebben-allergieen-invloed-op-uw-leven-of-dat-van-uw-kind~bd2b52a6/
https://www.demorgen.be/nieuws/op-de-amsterdamse-wallen-het-zijn-altijd-vrouwen-die-foto-s-maken-mannen-zijn-te-veel-bezig-met-wat-er-in-hun-broek-zit~bd638f25/
https://www.demorgen.be/beter-leven/van-koffie-over-suiker-tot-shoppen-deze-verslavingen-dringen-haast-ongemerkt-ons-leven-binnen~b8695295/
https://www.demorgen.be/snelnieuws/afscheid-van-tom-pintens-donderdag-in-olt-rivierenhof-weet-dat-tom-erg-van-gekleurde-sokken-hield~b4cc9bd7/
https://www.demorgen.be/meningen/van-9-tot-5~bbd332cb/
https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-aantal-slachtoffers-van-double-tap-raketaanval-op-pokrovsk-blijft-oplopen-zelensky-tegenoffensief-vordert-langzamer-dan-verwacht~b38bed0a/
https://www.demorgen.be/snelnieuws/italiaanse-premi

In [45]:
temp_links = links
for x in news_feed['url']:
    if x in temp_links:
        temp_links.remove(x)
for yyy in temp_links:
    print(yyy)

https://www.demorgen.be/meningen/van-9-tot-5~bbd332cb/
https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-aantal-slachtoffers-van-double-tap-raketaanval-op-pokrovsk-blijft-oplopen-zelensky-tegenoffensief-vordert-langzamer-dan-verwacht~b38bed0a/
https://www.demorgen.be/meningen/een-rib-uit-het-lijf~b650230e/
https://www.demorgen.be/snelnieuws/ons-kende-ons~b89e9d06/
https://www.demorgen.be/meningen/zak-cartoon-van-de-dag~b0de26ebb/
https://www.demorgen.be/tv-cultuur/heden-verse-vis~bb24d15f/


In [46]:
import os
import pymongo
MONGODB_URI = 'mongodb://localhost:27017'
database_name = "bouman_datatank"
collection_name = "articles"
client = pymongo.MongoClient(MONGODB_URI)
database = client[database_name]
collection = database[collection_name]

for article in news_feed_dict:
        article_url = article["url"]
        existing_article = collection.find_one({"url": article_url})
        if not existing_article:
            collection.insert_one(article)

client.close()

In [ ]:
MONGODB_URI = os.getenv("MONGODB_URI")
database_name = "bouman_datatank"
collection_name = "articles"
client = pymongo.MongoClient(MONGODB_URI)
database = client[database_name]
collection = database[collection_name]

for article in news_feed_dict:
        article_url = article["url"]
        existing_article = collection.find_one({"url": article_url})
        if not existing_article:
            collection.insert_one(article)

client.close()